### TODOS
- [ ] ensure no duplication from other datasets
- [ ] rewrite it as samantha dialog

In [1]:
# !wget https://huggingface.co/datasets/jondurbin/airoboros-gpt4-2.0/resolve/main/instructions.jsonl
# !mv instructions.jsonl ../cache_data/

In [2]:
import jsonlines as jsonl

with jsonl.open("../cache_data/instructions.jsonl") as f:
    data = list(f)

In [3]:
from datasets import Dataset

dataset = Dataset.from_list(data)
dataset

Dataset({
    features: ['instruction', 'response', 'category'],
    num_rows: 41087
})

In [4]:
set(dataset["category"])

{'agent',
 'card',
 'coding',
 'contextual',
 'cot',
 'counterfactual_contextual',
 'experience',
 'general',
 'joke',
 'orca',
 'plan',
 'puzzle',
 'quiz',
 'reasoning_or_math',
 'riddle',
 'roleplay',
 'song',
 'theory_of_mind',
 'trivia',
 'wordgame',
 'writing'}

In [16]:
dataset_pd = dataset.to_pandas()

dataset_pd.groupby("category").head(1)

,instruction,response,category
0,A logistics company is planning to build a new...,Step 1: Identify potential locations\nStart by...,reasoning_or_math
1,Simplify the concept of quantum physics and it...,Title: The Adventures of Quarky the Quantum Ca...,general
4,Create a speech about the importance of readin...,"Ladies and gentlemen, boys and girls,\nI've co...",roleplay
10,"What natural phenomenon, caused by the refract...",Rainbow,trivia
18,Develop a Golang application that interacts wi...,"package main\n\nimport (\n\t""database/sql""\n\t...",coding
24,"In a library, there are three sections: fictio...",Chris might assume that Dana and Erin are work...,theory_of_mind
25,"A red drum which sounds without being touched,...",The answer is a heart. The reasoning behind th...,riddle
33,"As an AI assistant, choose the appropriate fun...","function: csv_analytics\nparams:\n action: ""g...",agent
37,Do you know any jokes about winter accessories?,"A pair of mittens says to a hat, ""I'll stay he...",joke
38,Compose a sonnet about spring using ten words ...,"In spring, the world awakens from its winter's...",wordgame


In [5]:
dont_want = ["coding", "joke", "contextual", "orca", "counterfactual_contextual", "quiz"]
to_process = ["agent"]

dataset = dataset.filter(lambda row: row["category"] not in dont_want)

Filter:   0%|          | 0/41087 [00:00<?, ? examples/s]

In [6]:
dataset_normal = dataset.filter(lambda row: row["category"] not in to_process)
to_process_dataset = dataset.filter(lambda row: row["category"] in to_process)

Filter:   0%|          | 0/26714 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26714 [00:00<?, ? examples/s]

In [7]:
def to_chatml(row):
    person = dict(content=row["instruction"], role="user", name=None)
    me = dict(content=row["response"], role="assistant", name="AI")
    category = row["category"]
    situation_text = f"Person is talking to an intelligent AI system and giving instructions. AI assistant is following the instructions. It's a '{category}'-category instruction"
    situation = dict(content=situation_text, role="system", name="situation")

    return dict(chatml=[
        situation,
        person,
        me,
    ])

normal_dataset_chatml = dataset_normal.map(to_chatml).remove_columns(['instruction', 'response', 'category'])

Map:   0%|          | 0/25714 [00:00<?, ? examples/s]

In [8]:
#1 -> situation, role=system
#3 -> information, role=system, name=information
#2 - "Input: " -> person
# thought -> response, role=system, name=thought
# push to diwank/airoboros-agent

def to_chatml_2(row):
    instructions = row["instruction"].split("\n\n")
    situation = dict(content=instructions[0], role="system", name="situation")
    information = dict(content=instructions[2], role="system", name="information")
    person = dict(content="".join(instructions[1].split(":", 1)[1:]).strip(), role="user", name=None)
    thought = dict(content=row["response"], role="system", name="thought")

    return dict(
        chatml=[
            situation,
            information,
            person,
            thought,
        ]
    )

to_process_dataset_chatml = to_process_dataset.map(to_chatml_2).remove_columns(['instruction', 'response', 'category'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
normal_dataset_chatml.push_to_hub("diwank/airoboros-filtered", private=True)
to_process_dataset_chatml.push_to_hub("diwank/airoboros-agent", private=True)

Creating parquet from Arrow format: 100%|██████████████████████████████████████████████| 26/26 [00:00<00:00, 125.30ba/s]

Creating parquet from Arrow format: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 120.28ba/s]

Pushing dataset shards to the dataset hub: 100%|██████████████████████████████████████████| 1/1 [00:03<00:00,  3.80s/it]


In [10]:
to_process_dataset_chatml[0]

{'chatml': [{'content': "As an AI assistant, choose the appropriate function and its parameters from the given list according to the user's request. Please provide your answer in YAML format.",
   'name': 'situation',
   'role': 'system'},
  {'content': 'Available functions:\n- search:\n    description: Helps locate information by transforming the input query into relevant search terms and filters.\n    parameters:\n      search_terms: A list of crucial keywords or phrases from the input.\n      alternatives: A list of alternate important keywords or phrases from the input, like acronyms, common alternate synonyms, etc.\n    date_range:\n      begin: Restricts results to items dated on or after this value, if specified in the input query.\n      end: Restricts results to items dated on or before this value, if specified in the input query.\n- csv_analytics:\n    description: Useful for performing different operations like aggregation, counting, etc., on CSV data.\n    params:\n      ac